In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import Sequential
def printl(a):
    print(a)
    print('========================================================')

# CNN
## 卷积层实现
### 通过函数自定义权值
### `tf.nn.conv2d()`
在 TensorFlow 中，通过 tf.nn.conv2d 函数可以方便地实现 2D 卷积运算。tf.nn.conv2d 基于输入 $X: b ℎ 𝑤 𝑐_𝑖𝑛$ 和卷积核 $W: 𝑘 𝑘 𝑐_{𝑖𝑛} 𝑐_{𝑜𝑢𝑡}$ 进行卷积运算，得到输出$ O :b′ ℎ′ 𝑤′ 𝑐_{𝑜𝑢𝑡} $，其中$𝑐_{𝑖𝑛}$ 表示输入通道数，$𝑐_{𝑜𝑢𝑡}$ 表示卷积核的数量，也是输出特征图的通道数。

In [4]:
x = tf.random.normal([2,5,5,3]) # 模拟输入，3 通道，高宽为 5
# 需要根据[k,k,cin,cout]格式创建 W 张量，4 个 3x3 大小卷积核
w = tf.random.normal([3,3,3,4])
# 步长为 1, padding 为 0,
out = tf.nn.conv2d(x,w,strides=1,padding=[[0,0],[0,0],[0,0],[0,0]])
print(out.shape)

(2, 3, 3, 4)


其中 padding 参数的设置格式为：`padding=[[0,0],[上,下],[左,右],[0,0]]`  
例如，上下左右各 padding 一个单位，则 `padding=[[0,0],[1,1],[1,1],[0,0]]`  
通过设置参数 `padding='SAME'，strides=1` 可以直接得到输入、输出同大小的卷积层  
当s>1时，设置 padding='SAME'将使得输出高、宽将成为原来的$ \frac{1}{s}$

In [5]:
x = tf.random.normal([2,5,5,3]) # 模拟输入，3 通道，高宽为 5
w = tf.random.normal([3,3,3,4]) # 4 个 3x3 大小的卷积核
# 步长为,padding 设置为输出、输入同大小
# 需要注意的是, padding=same 只有在 strides=1 时才是同大小
out = tf.nn.conv2d(x,w,strides=1,padding='SAME')
print(out.shape)

(2, 5, 5, 4)


**bias的数目和卷积核的数目一样**  
卷积神经网络层与全连接层一样，可以设置网络带偏置向量。tf.nn.conv2d 函数是没有实现偏置向量计算的，添加偏置只需要手动累加偏置张量即可：

In [7]:
# 根据[cout]格式创建偏置向量
b = tf.zeros([4])
# 在卷积输出上叠加偏置向量，它会自动 broadcasting 为[b,h',w',cout]
out = out + b

### 通过类 `layers.Conv2D`

4 个 3x3 大小的卷积核的卷积层，步长为 1， padding 方案为'SAME'：

In [12]:
layer = tf.keras.layers.Conv2D(4,kernel_size=3,strides=1,padding='SAME')

4 个 3x4 大小的卷积核，竖直方向移动步长 $𝑠_ℎ = 2$，水平方向移动步长$𝑠_𝑤 = 1$：

In [13]:
layer = tf.keras.layers.Conv2D(4,kernel_size=(3,4),strides=(2,1),padding='SAME')

In [20]:
x = tf.random.truncated_normal([2,6,6,3])
layer = tf.keras.layers.Conv2D(4,kernel_size=3,strides=1,padding='SAME')
out = layer(x) # 前向计算
out.shape

TensorShape([2, 6, 6, 4])

In [21]:
layer.trainable_variables

[<tf.Variable 'conv2d_9/kernel:0' shape=(3, 3, 3, 4) dtype=float32, numpy=
 array([[[[-0.00560778,  0.01041681,  0.15193915,  0.1957666 ],
          [-0.03939977, -0.04343548,  0.108899  ,  0.10798538],
          [-0.10766666, -0.26234886,  0.02284598,  0.07559374]],
 
         [[-0.01738042,  0.19458124,  0.12696081, -0.19798371],
          [ 0.18966445, -0.18484047,  0.2535651 , -0.22216918],
          [ 0.2889981 ,  0.03528684, -0.12808494, -0.10228159]],
 
         [[-0.10554637,  0.12051079, -0.20063978, -0.23909077],
          [-0.2722934 ,  0.04857466, -0.12020154, -0.18777998],
          [-0.0919847 , -0.0076943 , -0.21867704,  0.1820271 ]]],
 
 
        [[[-0.01899943, -0.18684599, -0.11526744,  0.194927  ],
          [ 0.28169015, -0.04021236,  0.07959813, -0.06802872],
          [-0.01625314,  0.09175912, -0.13866606,  0.2556434 ]],
 
         [[-0.16311383, -0.08042434,  0.3028321 , -0.265151  ],
          [ 0.30112997,  0.03816772, -0.22044827,  0.01493108],
          [ 0.